In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras import Sequential
import pandas as pd
from keras.layers import Dense,SimpleRNN,Embedding,Flatten,LSTM
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
tokenizer = Tokenizer(oov_token='<nothing>')

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/sentiment_anlaysis_data/preprocessed_data_2.csv')

In [7]:
df = pd.read_csv('/content/drive/MyDrive/preprocessed_data_3.csv')

In [9]:
df.head()

,Unnamed: 0,text,sentiment
0,0,one review mention watch oz episod youll hook...,1
1,1,wonder littl product film techniqu unassum old...,1
2,2,thought wonder way spend time hot summer weeke...,1
3,3,basic there famili littl boy jake think there ...,0
4,4,petter mattei love time money visual stun film...,1


In [11]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
df.isna().sum()

text         4
sentiment    0
dtype: int64

In [14]:
df.dropna(inplace=True)

In [15]:
df.duplicated().sum()

316

In [16]:
df.drop_duplicates(inplace=True)

In [17]:
df.isna().sum()

text         0
sentiment    0
dtype: int64

In [18]:
df.duplicated().sum()

0

In [20]:
df.sentiment.value_counts()

1    34355
0    33374
Name: sentiment, dtype: int64

In [46]:
df.to_csv('preprocessed_data_4.csv')

In [19]:
tokenizer.fit_on_texts(df['text'])

In [21]:
sequences = tokenizer.texts_to_sequences(df['text'])

In [22]:
X_data = pad_sequences(sequences, padding='post', maxlen=100)

In [23]:
y_data = df['sentiment'].values

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [25]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=2, input_length=100))
model.add(LSTM(128,return_sequences=False))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 2)            20000     
                                                                 
 lstm (LSTM)                 (None, 128)               67072     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 87201 (340.63 KB)
Trainable params: 87201 (340.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_weights.h5', save_best_only=True, monitor='val_loss', mode='min')

In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train,epochs=100, batch_size = 32, validation_data=(X_test,y_test) , callbacks=[early_stop, checkpoint])

Epoch 1/100
1694/1694 [==============================] - 70s 40ms/step - loss: 0.1744 - acc: 0.9374 - val_loss: 0.4893 - val_acc: 0.8398
Epoch 2/100
   7/1694 [..............................] - ETA: 37s - loss: 0.1454 - acc: 0.9554

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1694/1694 [==============================] - 24s 14ms/step - loss: 0.1642 - acc: 0.9419 - val_loss: 0.4891 - val_acc: 0.8418
Epoch 3/100
1694/1694 [==============================] - 20s 12ms/step - loss: 0.1590 - acc: 0.9432 - val_loss: 0.5396 - val_acc: 0.8343
Epoch 4/100
1694/1694 [==============================] - 20s 12ms/step - loss: 0.1540 - acc: 0.9459 - val_loss: 0.4970 - val_acc: 0.8370
Epoch 5/100
1694/1694 [==============================] - 18s 11ms/step - loss: 0.1493 - acc: 0.9496 - val_loss: 0.5015 - val_acc: 0.8353


In [30]:
import pickle as pkl

In [31]:
with open('model.pkl', 'wb') as f:
    pkl.dump(model, f)


In [32]:
with open('tokenizer.pkl', 'wb') as f:
    pkl.dump(tokenizer, f)

In [33]:
loss, accuracy = model.evaluate(X_test, y_test)

424/424 [==============================] - 2s 5ms/step - loss: 0.4891 - acc: 0.8418


In [34]:
accuracy

0.8417983055114746

In [35]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
import re
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

stemmer = PorterStemmer()
stopwords = set(stopwords.words('english'))  # Ensure you've downloaded the stopwords set from nltk.

def preprocess(text):
    text = str(text).lower()
    text = re.sub('https?://\S+|www\.\S+','', text)  # URL removal
    text = re.sub('<.*?>+', '', text)  # HTML tags removal
    text = re.sub('\[.*?\]', '', text)  # Square brackets removal
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Punctuation removal
    text = re.sub('\n', '', text)  # Newline removal
    text = re.sub('\w*\d\w*', '', text)  # Words with digits removal
    text = ' '.join([word for word in text.split(' ') if word not in stopwords])  # Stopwords removal
    text = ' '.join([stemmer.stem(word) for word in text.split(' ')])  # Stemming
    sequence = tokenizer.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen=100)
    return sequence


In [45]:
test_data = 'I Hate You! Priyanka..!'
test_data = preprocess(test_data)
model.predict(test_data)

1/1 [==============================] - 0s 18ms/step


array([[0.11260258]], dtype=float32)